In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/test.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/data_description.txt


In [2]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor,VotingRegressor,StackingRegressor

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
np.random.seed(42)

In [5]:
train_data = pd.read_csv("../input/home-data-for-ml-course/train.csv")
test_data = pd.read_csv("../input/home-data-for-ml-course/test.csv")

In [6]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
train_data.describe(include='all')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1460.000000,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,...,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,...,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,1151,NaN,NaN,1454,50,925,1311,1459,...,NaN,3,157,49,NaN,NaN,NaN,1267,1198,NaN
mean,730.500000,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,...,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN,180921.195890
std,421.610009,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,...,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,34900.000000
25%,365.750000,20.000000,NaN,59.000000,7553.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN,NaN,129975.000000
50%,730.500000,50.000000,NaN,69.000000,9478.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,163000.000000
75%,1095.250000,70.000000,NaN,80.000000,11601.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,214000.000000


In [8]:
train_data.corr()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,1.000000,0.011156,-0.010601,-0.033226,-0.028365,0.012609,-0.012713,-0.021998,-0.050298,-0.005024,...,-0.029643,-0.000477,0.002889,-0.046635,0.001330,0.057044,-0.006242,0.021172,0.000712,-0.021917
MSSubClass,0.011156,1.000000,-0.386347,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.022936,-0.069836,...,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284
LotFrontage,-0.010601,-0.386347,1.000000,0.426095,0.251646,-0.059213,0.123349,0.088866,0.193458,0.233633,...,0.088521,0.151972,0.010700,0.070029,0.041383,0.206167,0.003368,0.011200,0.007450,0.351799
LotArea,-0.033226,-0.139781,0.426095,1.000000,0.105806,-0.005636,0.014228,0.013788,0.104160,0.214103,...,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843
OverallQual,-0.028365,0.032628,0.251646,0.105806,1.000000,-0.091932,0.572323,0.550684,0.411876,0.239666,...,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982
OverallCond,0.012609,-0.059316,-0.059213,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.128101,-0.046231,...,-0.003334,-0.032589,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856
YearBuilt,-0.012713,0.027850,0.123349,0.014228,0.572323,-0.375983,1.000000,0.592855,0.315707,0.249503,...,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897
YearRemodAdd,-0.021998,0.040581,0.088866,0.013788,0.550684,0.073741,0.592855,1.000000,0.179618,0.128451,...,0.205726,0.226298,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101
MasVnrArea,-0.050298,0.022936,0.193458,0.104160,0.411876,-0.128101,0.315707,0.179618,1.000000,0.264736,...,0.159718,0.125703,-0.110204,0.018796,0.061466,0.011723,-0.029815,-0.005965,-0.008201,0.477493
BsmtFinSF1,-0.005024,-0.069836,0.233633,0.214103,0.239666,-0.046231,0.249503,0.128451,0.264736,1.000000,...,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571,-0.015727,0.014359,0.386420


In [9]:
train_data.shape

(1460, 81)

In [10]:
X_train_full = train_data.drop(['SalePrice'],axis=1).copy()

In [11]:
X_train_full.shape

(1460, 80)

In [12]:
train_data.shape

(1460, 81)

In [13]:
y_train = train_data['SalePrice']

In [14]:
y_train.shape

(1460,)

In [15]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train)

In [16]:
X_train.shape,X_valid.shape

((1095, 80), (365, 80))

In [17]:
y_train.shape,y_valid.shape

((1095,), (365,))

In [18]:
for col in list(X_train.columns):
    if X_train[col].dtype == 'O':
        print("{},{}".format(X_train[col].name,X_train[col].nunique()))

MSZoning,5
Street,2
Alley,2
LotShape,4
LandContour,4
Utilities,2
LotConfig,5
LandSlope,3
Neighborhood,25
Condition1,9
Condition2,6
BldgType,5
HouseStyle,8
RoofStyle,6
RoofMatl,7
Exterior1st,15
Exterior2nd,16
MasVnrType,4
ExterQual,4
ExterCond,5
Foundation,6
BsmtQual,4
BsmtCond,4
BsmtExposure,4
BsmtFinType1,6
BsmtFinType2,6
Heating,6
HeatingQC,5
CentralAir,2
Electrical,4
KitchenQual,4
Functional,7
FireplaceQu,5
GarageType,6
GarageFinish,3
GarageQual,5
GarageCond,5
PavedDrive,3
PoolQC,3
Fence,4
MiscFeature,4
SaleType,9
SaleCondition,6


In [19]:
cat=[i for i in list(X_train.columns) if X_train[i].dtype == 'O' and X_train[i].nunique()<=4]
num=[i for i in list(X_train.columns) if X_train[i].dtype in ['int64','float64'] ]

In [20]:
for col in cat:
    print("{},{}".format(X_train[col].name,X_train[col].nunique()))

Street,2
Alley,2
LotShape,4
LandContour,4
Utilities,2
LandSlope,3
MasVnrType,4
ExterQual,4
BsmtQual,4
BsmtCond,4
BsmtExposure,4
CentralAir,2
Electrical,4
KitchenQual,4
GarageFinish,3
PavedDrive,3
PoolQC,3
Fence,4
MiscFeature,4


In [21]:
len(num),len(cat)

(37, 19)

In [22]:
sum([X_train[i].nunique() for i in list(X_train.columns) if X_train[i].dtype == 'O' and X_train[i].nunique()<=4])

64

In [23]:
num_cleaning = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy='median')),
    ("scaling",StandardScaler())
])

In [24]:
cat_cleaning = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy='most_frequent')),
    ("Encoder",OneHotEncoder(handle_unknown='ignore'))
])

In [25]:
preprocessor = ColumnTransformer(transformers=[
    ('num',num_cleaning,num),
    ('cat',cat_cleaning,cat)
])

In [26]:
xgb_reg = Pipeline(steps=[
    ('preprocessing',preprocessor),
    ('xgboost',XGBRegressor(max_depth=4,learning_rate=0.1,n_estimators=100))
])

In [27]:
xgb_reg.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                   

In [28]:
xgb_reg.score(X_train,y_train),xgb_reg.score(X_valid,y_valid)

(0.9798872440378431, 0.8884976209566269)

In [29]:
RF_reg = Pipeline(steps=[
    ('preprocessing',preprocessor),
    ('randomForest',RandomForestRegressor())
])
RF_reg.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                   

In [30]:
RF_reg.score(X_train,y_train),xgb_reg.score(X_valid,y_valid)

(0.978232366323128, 0.8884976209566269)

In [31]:
GB_reg = Pipeline(steps=[
    ('preprocessing',preprocessor),
    ('Gradientboosting',GradientBoostingRegressor())
])
GB_reg.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                   

In [32]:
GB_reg.score(X_train,y_train),GB_reg.score(X_valid,y_valid)

(0.9664455989098105, 0.8946333893747502)

In [33]:
xgbv_reg = XGBRegressor(max_depth=4,learning_rate=0.1,n_estimators=100)
RFv_reg = RandomForestRegressor()
GBv_reg = GradientBoostingRegressor()

votingv_reg = VotingRegressor(
    estimators=[('xgb', xgbv_reg), ('rf', RFv_reg), ('GB', GBv_reg)])

In [34]:
voting_reg = Pipeline(steps=[
    ('preprocessing',preprocessor),
    ('Voting',votingv_reg)
])
voting_reg.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                   

In [35]:
voting_reg.score(X_train,y_train),voting_reg.score(X_valid,y_valid)

(0.9790613800352967, 0.8983947376269402)

In [36]:
y_pred = voting_reg.predict(test_data)

In [37]:
y_pred.shape,test_data.shape

((1459,), (1459, 80))

In [38]:
output = pd.DataFrame({'Id': test_data.Id,'SalePrice': y_pred})
output.to_csv('submission.csv', index=False)

In [39]:
outputs = pd.read_csv('./submission.csv')

In [40]:
output.head()

,Id,SalePrice
0,1461,128292.439518
1,1462,151441.515979
2,1463,178631.585492
3,1464,185428.088392
4,1465,207217.237850
